<a href="https://colab.research.google.com/github/kowalski-k/projekt_TOM/blob/master/Projekt_TOM_U_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importowanie używanych bibliotek

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

#sprawdzenie używanej wersji tensorflow
print(tf.__version__)

# Pobranie danych i zapoznanie się z nimi

In [0]:
#pobieranie danych
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
! git clone https://github.com/neheller/kits19.git 
%cd kits19
! python -m starter_code.get_imaging
!git checkout feature/google-colab
!make

In [0]:
#wczytanie danych dla jednego wybranego pacjenta
from starter_code.utils import load_case
volume, segmentation = load_case("case_00113")

In [0]:
#wizualizacja nowotworów i nerek na tle obrazu CT (zobaczyć jedynie jak działa, przyda się na końcu)
'''
visualize("case_00113", "case_113")
''' 

In [0]:
#wyodrębnienie i obrazowanie segementation
segmentation = segmentation.get_fdata()
(a,x,y) = segmentation.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(segmentation[i],'gray')
  plt.title(i)

In [0]:
#wyodrębnienie i wczytywanie "czystych" obrazow bez zaznaczen (volume)
volume = volume.get_fdata()
(a,x,y) = volume.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(volume[i],'gray')
  plt.title(i)

In [0]:
#zdobycie przydatnych w późniejszej części informacji o segmentation
i = 19
print(segmentation.shape)
print(type(segmentation[i]))
print(np.amax(segmentation[i]))
print(np.amin(segmentation[i]))
print(segmentation[i])
plt.imshow(segmentation[i],'gray')

In [0]:
#zdobycie przydatnych w późniejszej części informacji o volume
print(volume.shape)
print(type(volume[0]))
print(np.amax(volume[0]))
print(np.amin(volume[0]))
print(volume[0])
plt.imshow(volume[0],'gray')

# Przygotowanie danych do trenowania


In [0]:
#pierwsze 210 pacjentow ma posegmentowane nerki i nowotwory. Do trenowania uzyjemy jedynie 5 case'ów ze względu na to,
#że przy próbie wczytania większej ilości występuje błąd związany z brakiem odpowiedniej ilości pamięci RAM

all_volumes = []
all_segmentations = []

for i in range(5):
  volume, segmentation = load_case(i+5)
  segmentation = segmentation.get_fdata()
  volume = volume.get_fdata()
  all_volumes.append(volume)
  all_segmentations.append(segmentation)

In [0]:
#wczytanie obrazow bez oznaczen do jednej listy
volumes = []

for one_volume in all_volumes:
  (a,y,x) = one_volume.shape
  for i in range(0,a,1):
    volumes.append(one_volume[i])

In [0]:
#wczytanie masek dla nowotworow do jednej listy 

kidney = []
tumor = []

numbers_of_slice_kidney = []
numbers_of_slice_tumor = []



for one_segmentation in all_segmentations:
  (a,y,x) = one_segmentation.shape
  for i in range(0,a,1):
    for y_col in range(0,y,1):
      for x_row in range(0,x,1):
        if (one_segmentation[i][y_col][x_row] == 2):
          tumor.append(one_segmentation[i])
          numbers_of_slice_tumor.append(i)
          break
      else:
        continue
      break

In [0]:
#wczytanie masek dla nerek do jednej

for one_segmentation in all_segmentations:
  (a,y,x) = one_segmentation.shape
  for i in range(0,a,1):
    for y_col in range(0,y,1):
      for x_row in range(0,x,1):
        if (one_segmentation[i][y_col][x_row] == 1):
          kidney.append(one_segmentation[i])
          numbers_of_slice_kidney.append(i)
          break
      else:
        continue
      break

In [0]:
#wyznaczenie masek tylko dla wartosci nerek albo tylko wartosci nowotworow
#usunieto wartosci dla nerki w masce nowotworow i odwrotnie dla maski nerek 

for i in range(len(kidney)):
  for y_col in range(len(kidney[i])):
    for x_row in range(len(kidney[i][y_col])):
      if (kidney[i][y_col][x_row] == 2):
        kidney[i][y_col][x_row] = 0


for i in range(len(tumor)):
  for y_col in range(len(tumor[i])):
    for x_row in range(len(tumor[i][y_col])):
      if (tumor[i][y_col][x_row] == 1):
        tumor[i][y_col][x_row] = 0
      elif (tumor[i][y_col][x_row] == 2):
        tumor[i][y_col][x_row] = 1

In [0]:
#przygotowanie macierzy na trening
X_train = []
Y_train = []

#uzupełenie macierzy obrazami + zamiana rozmiaru obrazów z (512,512) na (512,512,1)
for i in range(len(kidney)):
  #volume
  img_volume = volumes[numbers_of_slice_kidney[i]]
  img_volume_new_shape = np.expand_dims(img_volume,axis=-1)
  X_train.append(img_volume_new_shape)
  #kidney
  img_kindey = kidney[i]
  img_kidney_new_shape = np.expand_dims(img_kindey,axis=-1)
  Y_train.append(img_kidney_new_shape)

#bez normalizacji
X_train = np.asarray(X_train)

#wybrane przekroje (volume) należy znormalizować do wartości [0,1]
for i in range(len(X_train)):
  X_train[i] = (X_train[i]-np.amin(X_train[i]))/(np.amax(X_train[i])-np.amin(X_train[i]))

#finalne przygotowane macierze do treningu
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

#informacje
print(X_train.shape)
print(Y_train.shape)
print(numbers_of_slice_kidney)
print(numbers_of_slice_tumor)

# Architektura U-Net

In [0]:
#obraz wejsciowy 
image_width = 512
image_height = 512
image_channels = 1 
inputs = tf.keras.layers.Input((image_width,image_height,image_channels))

#sciezka skurczu
#1
convolution1 = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(inputs)
convolution1 = tf.keras.layers.Dropout(0.1)(convolution1)
convolution1 = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution1)
max_pooling1 = tf.keras.layers.MaxPooling2D((2,2))(convolution1)
#2
convolution2 = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(max_pooling1)
convolution2 = tf.keras.layers.Dropout(0.1)(convolution2)
convolution2 = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution2)
max_pooling2 = tf.keras.layers.MaxPooling2D((2,2))(convolution2)
#3
convolution3 = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(max_pooling2)
convolution3 = tf.keras.layers.Dropout(0.1)(convolution3)
convolution3 = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution3)
max_pooling3 = tf.keras.layers.MaxPooling2D((2,2))(convolution3)
#4
convolution4 = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(max_pooling3)
convolution4 = tf.keras.layers.Dropout(0.1)(convolution4)
convolution4 = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution4)
max_pooling4 = tf.keras.layers.MaxPooling2D((2,2))(convolution4)
#5 - ostatnia (bez MaxPooling)
convolution5 = tf.keras.layers.Conv2D(1024, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(max_pooling4)
convolution5 = tf.keras.layers.Dropout(0.1)(convolution5)
convolution5 = tf.keras.layers.Conv2D(1024, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution5)

#sciezka ekspansji
#6 
up_convolution6 = tf.keras.layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same')(convolution5)
up_convolution6 = tf.keras.layers.concatenate([up_convolution6, convolution4])
convolution6 = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(up_convolution6)
convolution6 = tf.keras.layers.Dropout(0.1)(convolution6)
convolution6 = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution6)
#7
up_convolution7 = tf.keras.layers.Conv2DTranspose(256, (3,3), strides=(2,2), padding='same')(convolution6)
up_convolution7 = tf.keras.layers.concatenate([up_convolution7, convolution3])
convolution7 = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(up_convolution7)
convolution7 = tf.keras.layers.Dropout(0.1)(convolution7)
convolution7 = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution7)
#8
up_convolution8 = tf.keras.layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(convolution7)
up_convolution8 = tf.keras.layers.concatenate([up_convolution8, convolution2])
convolution8 = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(up_convolution8)
convolution8 = tf.keras.layers.Dropout(0.1)(convolution8)
convolution8 = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution8)
#9
up_convolution9 = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same')(convolution8)
up_convolution9 = tf.keras.layers.concatenate([up_convolution9, convolution1], axis=3)
convolution9 = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(up_convolution9)
convolution9 = tf.keras.layers.Dropout(0.1)(convolution9)
convolution9 = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(convolution9)

outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(convolution9)

In [0]:
#optymalizacja
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.summary()

# Trenowanie sieci do wykrywania nerek

In [0]:
#dodatkowe paramtery do szybszego kończenia uczenia
callbacks = [
             tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss', mode='min'),
             tf.keras.callbacks.TensorBoard(log_dir='logs'),
             tf.keras.callbacks.ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
             ]

In [0]:
#nauka (batch_size tylko 1, bo na więcej nie staracza RAMu)
results = model.fit(X_train , Y_train, validation_split=0.1, verbose=1, epochs=5, batch_size = 1, callbacks = callbacks)

# Testowanie sieci

In [0]:
#załadowanie obrazów testowych - X_test

In [0]:
#przewidywanie
#prediction = model.predict(X_test, batch_size=1 , verbose=1)
#zakładamy, że prawdopodobieństwo powyżej 50% pozwala na dopasowanie piksela do klasy
#prediction_bin = (preds_val > 0.5)

In [0]:
#wizualizacja 
#przywrócenie do wejściowego kształtu
#prediction_reshaped = np.reshape(prediction_bin,(512,512))
#zobrazowwanie prediction
#plt.figure()
#plt.imshow(prediction_reshaped,'gray')

#obecnie za małe prawdopodobieństwa (max.10%), trzeba popracować, żeby sieć była mądrzejsza 